# How to use the kube client in api server
The existing workbench leveraged `ReplicationController`(https://kubernetes.io/docs/concepts/workloads/controllers/replicationcontroller/). At this point, a `Deployment` that configures a `ReplicaSet` is now the recommended way to set up replication.
https://kubernetes.io/docs/concepts/workloads/controllers/deployment/
- [Access Clusters Using the Kubernetes API](https://kubernetes.io/docs/tasks/administer-cluster/access-cluster-api/)
- [Python Client Examples](https://github.com/kubernetes-client/python/tree/master/examples)


## Check running kube cluster

In [9]:
from kubernetes import client, config

# Configs can be set in Configuration class directly or using helper utility
config.load_kube_config()

v1 = client.CoreV1Api()
print("Listing pods with their IPs:")
ret = v1.list_pod_for_all_namespaces(watch=False)
for i in ret.items:
    print("%s\t%s\t%s\t%s" % (i.status.pod_ip, i.status.phase, i.metadata.namespace, i.metadata.name))

Listing pods with their IPs:
172.17.0.3	Running	default	busybox-test
172.17.0.8	Running	default	hello-minikube
172.17.0.7	Running	default	wordpress-c76fbfd95-cqg4p
172.17.0.10	Running	default	wordpress-mysql-b5d56c447-v2d8d
172.17.0.2	Running	kube-system	coredns-74ff55c5b-l6t4s
192.168.64.4	Running	kube-system	etcd-minikube
172.17.0.9	Running	kube-system	ingress-nginx-controller-65cf89dc4f-s2t59
192.168.64.4	Running	kube-system	kube-apiserver-minikube
192.168.64.4	Running	kube-system	kube-controller-manager-minikube
192.168.64.4	Running	kube-system	kube-proxy-swdlv
192.168.64.4	Running	kube-system	kube-scheduler-minikube
172.17.0.4	Running	kube-system	nfs-provisioner-76b9b4576-d5rkh
192.168.64.4	Running	kube-system	storage-provisioner
172.17.0.6	Running	kubernetes-dashboard	dashboard-metrics-scraper-f6647bd8c-cxbmn
172.17.0.5	Running	kubernetes-dashboard	kubernetes-dashboard-968bcb79-gc4xp


In [2]:
from kubernetes import client, config


def main():
    # Configs can be set in Configuration class directly or using helper
    # utility. If no argument provided, the config will be loaded from
    # default location.
    config.load_kube_config()

    print("Supported APIs (* is preferred version):")
    print("%-40s %s" %
          ("core", ",".join(client.CoreApi().get_api_versions().versions)))
    for api in client.ApisApi().get_api_versions().groups:
        versions = []
        for v in api.versions:
            name = ""
            if v.version == api.preferred_version.version and len(
                    api.versions) > 1:
                name += "*"
            name += v.version
            versions.append(name)
        print("%-40s %s" % (api.name, ",".join(versions)))


if __name__ == '__main__':
    main()

Supported APIs (* is preferred version):
core                                     v1
apiregistration.k8s.io                   *v1,v1beta1
apps                                     v1
events.k8s.io                            *v1,v1beta1
authentication.k8s.io                    *v1,v1beta1
authorization.k8s.io                     *v1,v1beta1
autoscaling                              *v1,v2beta1,v2beta2
batch                                    *v1,v1beta1
certificates.k8s.io                      *v1,v1beta1
networking.k8s.io                        *v1,v1beta1
extensions                               v1beta1
policy                                   v1beta1
rbac.authorization.k8s.io                *v1,v1beta1
storage.k8s.io                           *v1,v1beta1
admissionregistration.k8s.io             *v1,v1beta1
apiextensions.k8s.io                     *v1,v1beta1
scheduling.k8s.io                        *v1,v1beta1
coordination.k8s.io                      *v1,v1beta1
node.k8s.io            

## How to create a Deployment
- https://github.com/kubernetes-client/python/blob/master/examples/notebooks/create_deployment.ipynb

### 1. Load config

In [60]:
from kubernetes import client, config

LABELS = {"app": "nginx"}
DEPLOYMENT_NAME = "nginx-deployment"

# Load config from default location
config.load_kube_config()
apps_api = client.AppsV1Api()

apps_api

### 2. Pod container

In [61]:
# Configure Pod template container
container = client.V1Container(
    name = "nginx",
    image="nginx:1.15.4",
    ports=[client.V1ContainerPort(container_port=80)],
    resources=client.V1ResourceRequirements(
        requests={"cpu": "100m", "memory":"200Mi"},
        limits={"cpu": "500m", "memory":"500Mi"},
    ),
)

#container

### 3. Spec section

In [62]:
template = client.V1PodTemplateSpec(
    metadata=client.V1ObjectMeta(labels=LABELS),
    spec=client.V1PodSpec(containers=[container]),
)

#template

### 4. Specification of deployment

In [64]:
spec = client.V1DeploymentSpec(
    replicas=1,
    template=template,
    selector=client.V1LabelSelector(match_labels=LABELS)
)

#spec

### 5. Instantiate the deployment object

In [65]:
deployment = client.V1Deployment(
    api_version="apps/v1",
    kind="Deployment",
    metadata=client.V1ObjectMeta(name=DEPLOYMENT_NAME),
    spec=spec,
)

#deployment

### 6. Create deployment

In [66]:
def create_deployment(api, deployment):
    # Create deployement
    resp = api.create_namespaced_deployment(
        body=deployment, namespace="default"
    )

    print("\n[INFO] deployment `nginx-deployment` created.\n")
    print("%s\t%s\t\t\t%s\t%s" % ("NAMESPACE", "NAME", "REVISION", "IMAGE"))
    print(
        "%s\t\t%s\t%s\t\t%s\n"
        % (
            resp.metadata.namespace,
            resp.metadata.name,
            resp.metadata.generation,
            resp.spec.template.spec.containers[0].image,
        )
    )


In [68]:
create_deployment(apps_api, deployment)


[INFO] deployment `nginx-deployment` created.

NAMESPACE	NAME			REVISION	IMAGE
default		nginx-deployment	1		nginx:1.15.4



### 7. Delete deployment

In [56]:
def delete_deployment(api):
    # Delete deployment
    resp = api.delete_namespaced_deployment(
        name=DEPLOYMENT_NAME,
        namespace="default",
        body=client.V1DeleteOptions(
            propagation_policy="Foreground", grace_period_seconds=5
        ),
    )
    print("\n[INFO] deployment `nginx-deployment` deleted.")


In [59]:
delete_deployment(apps_api)


[INFO] deployment `nginx-deployment` deleted.


## How to create Service
- https://github.com/kubernetes-client/python/blob/master/examples/notebooks/create_service.ipynb
### 1. Load config

In [117]:
from kubernetes import client, config

LABELS = {"app": "nginx"}
SERVICE_NAME = "nginx-service"

config.load_kube_config()
api_instance = client.CoreV1Api()
service = client.V1Service()

service

{'api_version': None,
 'kind': None,
 'metadata': None,
 'spec': None,
 'status': None}

### 2. Create API resource instance with Service .spec description
We define `NodePort` to expose the service.Then we can access this service with `http://<minikube ip>:<node_port>`.

For example, run the following command in the terminal running minikube.
```
% minikube ip
192.168.64.4
```
Then you can access `http://192.168.64.4:32000/` 

In [142]:
service.api_version = "v1"
service.kind = "Service"
service.metadata = client.V1ObjectMeta(name=SERVICE_NAME)

spec = client.V1ServiceSpec()
spec.selector = LABELS
spec.type = "NodePort"
spec.ports = [client.V1ServicePort(protocol="TCP", port=8080, target_port=80, node_port=32000)]
service.spec = spec

service

{'api_version': 'v1',
 'kind': 'Service',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': None,
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': None,
              'name': 'nginx-service',
              'namespace': None,
              'owner_references': None,
              'resource_version': None,
              'self_link': None,
              'uid': None},
 'spec': {'cluster_ip': None,
          'external_i_ps': None,
          'external_name': None,
          'external_traffic_policy': None,
          'health_check_node_port': None,
          'ip_family': None,
          'load_balancer_ip': None,
          'load_balancer_source_ranges': None,
          'ports': [{'name': None,
                     'node_port': 32

### 3. Create Service

In [141]:
api_instance.create_namespaced_service(namespace="default", body=service)

{'api_version': 'v1',
 'kind': 'Service',
 'metadata': {'annotations': None,
              'cluster_name': None,
              'creation_timestamp': datetime.datetime(2021, 5, 24, 22, 42, 18, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': [{'api_version': 'v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:spec': {'f:externalTrafficPolicy': {},
                                                           'f:ports': {'.': {},
                                                                       'k:{"port":8080,"protocol":"TCP"}': {'.': {},
                                                                                                            'f:nodePort': {},
                                

### 4. Delete Service

In [137]:
api_instance.delete_namespaced_service(name=SERVICE_NAME, namespace="default")

{'api_version': 'v1',
 'code': None,
 'details': {'causes': None,
             'group': None,
             'kind': 'services',
             'name': 'nginx-service',
             'retry_after_seconds': None,
             'uid': '120f4dc7-4379-463f-806c-be178275fe63'},
 'kind': 'Status',
 'message': None,
 'metadata': {'_continue': None,
              'remaining_item_count': None,
              'resource_version': None,
              'self_link': None},
 'reason': None,
 'status': 'Success'}

## Dependency
- clowder - mongo
- need to prepare volumn?

In [7]:
import json

with open('mongo.json') as json_file:
    data = json.load(json_file)

data

{'key': 'mongo',
 'label': 'MongoDB',
 'image': {'name': 'mongo', 'tags': ['3.2.4']},
 'description': 'A cross-platform document-oriented NoSQL database',
 'display': 'standalone',
 'access': 'internal',
 'ports': [{'port': 27017, 'protocol': 'tcp'}],
 'volumeMounts': [{'mountPath': '/data/db'}],
 'resourceLimits': {'cpuMax': 500,
  'cpuDefault': 100,
  'memMax': 250,
  'memDefault': 100},
 'readinessProbe': {'type': 'tcp',
  'path': '',
  'port': 27017,
  'initialDelay': 10,
  'timeout': 60},
 'repositories': [{'type': 'git', 'url': 'https://github.com/mongodb/mongo'}],
 'logo': '/asset/png/logos/mongodb-logo.jpg',
 'info': 'https://nationaldataservice.atlassian.net/wiki/display/NDSC/MongoDB',
 'tags': ['24']}

In [8]:
with open('clowder.json') as json_file:
    data = json.load(json_file)

data

{'label': 'Clowder',
 'key': 'clowder',
 'description': 'A scalable data repository where you can share, organize and analyze data.',
 'image': {'name': 'clowder/clowder',
  'tags': ['1.1.1',
   '1.0.1',
   '1.0',
   '0.9.5',
   '0.9.4.2',
   '0.9.4.1',
   '0.9.4',
   'latest']},
 'display': 'stack',
 'access': 'external',
 'logo': '/asset/png/logos/clowder-logo.png',
 'depends': [{'key': 'mongo', 'required': True},
  {'key': 'rabbitmq', 'required': False},
  {'key': 'elasticsearch', 'required': False},
  {'key': 'imagepreview', 'required': False},
  {'key': 'imagemetadata', 'required': False},
  {'key': 'pdfpreview', 'required': False},
  {'key': 'audiopreview', 'required': False},
  {'key': 'speech2text', 'required': False},
  {'key': 'videopreview', 'required': False},
  {'key': 'plantcv', 'required': False},
  {'key': 'digestextractor', 'required': False},
  {'key': 'officepreview', 'required': False}],
 'config': [{'name': 'SMTP_HOST',
   'value': 'outbound.ucsd.edu',
   'canOverr

https://github.com/kubernetes-client/python/blob/master/kubernetes/README.md